In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from pathlib import Path
import os
DATA_DIR=Path('/kaggle/input')
for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3c-shared-task-purpose/test.csv
/kaggle/input/3c-shared-task-purpose/sample_submission.csv
/kaggle/input/3c-shared-task-purpose/train.csv
/kaggle/input/3c-shared-task-influence/test.csv
/kaggle/input/3c-shared-task-influence/sample_submission.csv
/kaggle/input/3c-shared-task-influence/train.csv


In [2]:
TRAIN_PATH = DATA_DIR / "3c-shared-task-influence" / "train.csv"
TEST_PATH = DATA_DIR / "3c-shared-task-influence" / "test.csv"

In [3]:
! head /kaggle/input/3c-shared-task-influence/sample_submission.csv

In [4]:
import pandas as pd

In [5]:
df_train = pd.read_csv(str(TRAIN_PATH).replace("influence", "purpose"))
df_train.columns

Index(['unique_id', 'core_id', 'citing_title', 'citing_author', 'cited_title',
       'cited_author', 'citation_context', 'citation_class_label'],
      dtype='object')

In [6]:
df_train.head()

,unique_id,core_id,citing_title,citing_author,cited_title,cited_author,citation_context,citation_class_label
0,CC1,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Web search personalization with ontological us...,Sieg,They usually generate user models that describ...,5
1,CC2,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Exploring Scholarly Data with Rexplore,Osborne,The Computer Science Ontology (CSO)[3]is a lar...,0
2,CC3,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Klink-2: Integrating Multiple Web Sources to G...,Osborne,"In order to do so, we characterized all SN pub...",0
3,CC4,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Forecasting the Spreading of Technologies in R...,Osborne,"This API supports a number of applications, in...",0
4,CC5,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Supporting Springer Nature Editors by means of...,Osborne,It works according to three main steps:1) It r...,5


In [7]:
df_train = pd.read_csv(TRAIN_PATH).merge(
    pd.read_csv(str(TRAIN_PATH).replace("influence", "purpose"))[["unique_id", "citation_class_label"]],
    on="unique_id"
)
df_train.head()

,unique_id,core_id,citing_title,citing_author,cited_title,cited_author,citation_context,citation_influence_label,citation_class_label
0,CC1,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Web search personalization with ontological us...,Sieg,They usually generate user models that describ...,0,5
1,CC2,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Exploring Scholarly Data with Rexplore,Osborne,The Computer Science Ontology (CSO)[3]is a lar...,0,0
2,CC3,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Klink-2: Integrating Multiple Web Sources to G...,Osborne,"In order to do so, we characterized all SN pub...",0,0
3,CC4,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Forecasting the Spreading of Technologies in R...,Osborne,"This API supports a number of applications, in...",1,0
4,CC5,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Supporting Springer Nature Editors by means of...,Osborne,It works according to three main steps:1) It r...,1,5


In [8]:
df_test = pd.read_csv(TEST_PATH).merge(
    pd.read_csv(str(TEST_PATH).replace("influence", "purpose"))[["unique_id"]],
    on="unique_id"
)
df_test.head()

,unique_id,core_id,citing_title,citing_author,cited_title,cited_author,citation_context
0,CCT1,81153632,"Two steps back, one step forward: reconstructi...",Severin Hohensinner,The English village community examined in its ...,Seebohm,"In historical research, regressive methods mov..."
1,CCT2,81153632,"Two steps back, one step forward: reconstructi...",Severin Hohensinner,Die italienischen Begr nder der Wiener Donauka...,Slezak,"For example, a new cut-off channel at the Dona..."
2,CCT3,81153632,"Two steps back, one step forward: reconstructi...",Severin Hohensinner,Wien im Bild historischer Karten Die Entwicklu...,Opll,It can be considered as the first map of the V...
3,CCT4,81153632,"Two steps back, one step forward: reconstructi...",Severin Hohensinner,The hydraulic geometry of stream channels and ...,Leopold,According to the ''hydraulic geometry'' approa...
4,CCT5,81153632,"Two steps back, one step forward: reconstructi...",Severin Hohensinner,From memory to written record,Clanchy,Historians ask for the motivation and interest...


In [9]:
df = pd.concat([
    df_train.assign(split="train"),
    df_test.assign(split="test"),
], axis=0, sort=False).reset_index(drop=True)
df.head()

,unique_id,core_id,citing_title,citing_author,cited_title,cited_author,citation_context,citation_influence_label,citation_class_label,split
0,CC1,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Web search personalization with ontological us...,Sieg,They usually generate user models that describ...,0.0,5.0,train
1,CC2,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Exploring Scholarly Data with Rexplore,Osborne,The Computer Science Ontology (CSO)[3]is a lar...,0.0,0.0,train
2,CC3,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Klink-2: Integrating Multiple Web Sources to G...,Osborne,"In order to do so, we characterized all SN pub...",0.0,0.0,train
3,CC4,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Forecasting the Spreading of Technologies in R...,Osborne,"This API supports a number of applications, in...",1.0,0.0,train
4,CC5,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Supporting Springer Nature Editors by means of...,Osborne,It works according to three main steps:1) It r...,1.0,5.0,train


In [10]:
df.split.value_counts()

train    3000
test     1000
Name: split, dtype: int64

In [11]:
df.pivot_table(index="citation_influence_label", columns="split", values="unique_id", aggfunc=len).sort_values("train", ascending=False)

split,train
citation_influence_label,
0.0,1568
1.0,1432


In [12]:
df.pivot_table(index="citation_class_label", columns="split", values="unique_id", aggfunc=len).sort_values("train", ascending=False)

split,train
citation_class_label,
0.0,1648
5.0,475
1.0,368
4.0,276
2.0,171
3.0,62


In [13]:
df.pivot_table(index="citing_author", columns="split", values="unique_id", aggfunc=len).sort_values("train", ascending=False)

split,test,train
citing_author,,
Geraldine Gray,NaN,120.0
Fabio Reale,NaN,93.0
Eric Klinger,NaN,86.0
Ifigeneia Mavranezouli,NaN,85.0
Nick Byrd,NaN,84.0
...,...,...
Thomas Elmqvist,35.0,NaN
Thomas Toulias,1.0,NaN
Timo Partonen,24.0,NaN


In [14]:
df.pivot_table(index="cited_author", columns="split", values="unique_id", aggfunc=len).sort_values("train", ascending=False)

split,test,train
cited_author,,
Klinger,NaN,10.0
Anderson,NaN,10.0
Chen,3.0,9.0
Wang,4.0,9.0
Sasakura,NaN,9.0
...,...,...
ter Steege,2.0,NaN
van Deth,1.0,NaN
van Loon,1.0,NaN


In [15]:
df.core_id.value_counts()

93144659      120
81800077       93
82852800       86
54847651       84
81068384       74
             ... 
2791771689      1
10182392        1
1942026013      1
2185862         1
82162478        1
Name: core_id, Length: 227, dtype: int64

In [16]:
import requests
import json
import time

In [17]:
CORE_REQUEST_URL="https://core.ac.uk:443/api-v2/articles/get?metadata=true&fulltext=true&citations=true&similar=false&duplicate=false&urls=false&faithfulMetadata=false&apiKey=gHQKEJ9bMBe7FntNCXqjrASuz0xc5dTU"
def get_paper_data(core_ids, batch_size=10):
    core_ids = list(core_ids)
    for i in range(0, len(core_ids), batch_size):
        batch = core_ids[i:i+batch_size]
        resp = requests.post(CORE_REQUEST_URL, json=batch)
        batch_resp_json = resp.json()
        yield batch_resp_json
        print(f"Found {len(batch_resp_json)} responses. Sleeping for 2 seconds.")
        time.sleep(2)

In [18]:
%%time
CORE_DATA_PATH=Path("./core_data.jsonl")
CORE_DATA_URL="https://gist.githubusercontent.com/napsternxg/f514e8ac039998e129ad187956b7eb9f/raw/00e34089890fd158f0867e1b3436257a511d040f/core_data.jsonl"
COLLECT_CORE_DATA=False
if not CORE_DATA_PATH.exists():
    print(f"File {CORE_DATA_PATH} does not exists.")
    if COLLECT_CORE_DATA:
        print(f"Collecting data via the CORE API")
        core_data = sum(get_paper_data(df.core_id.unique().tolist(), batch_size=10), [])
    else:
        print(f"Fetching {CORE_DATA_URL}.")
        resp = requests.get(CORE_DATA_URL).text
        core_data = []
        for line in resp.splitlines():
            d = json.loads(line)
            core_data.append(d)
        print(f"Found {len(core_data)} records.")
    print(f"Found {len(core_data)} records. Writing to {CORE_DATA_PATH}")
    with open(CORE_DATA_PATH, "w+") as fp:
        for d in core_data:
            print(json.dumps(d), file=fp)
else:
    print(f"File {CORE_DATA_PATH} exists. Loading.")
    with open(CORE_DATA_PATH) as fp:
        core_data = []
        for line in fp:
            d = json.loads(line)
            core_data.append(d)
    print(f"Found {len(core_data)} records.")

File core_data.jsonl does not exists.
Fetching https://gist.githubusercontent.com/napsternxg/f514e8ac039998e129ad187956b7eb9f/raw/00e34089890fd158f0867e1b3436257a511d040f/core_data.jsonl.
Found 227 records.
Found 227 records. Writing to core_data.jsonl
CPU times: user 247 ms, sys: 160 ms, total: 407 ms
Wall time: 1.55 s


In [19]:
core_data[0]["data"]["year"]

2018

In [20]:
found_keys = [
    int(d["data"]["id"])
    for d in core_data
    if d["status"] == "OK"
]
len(found_keys)

176

In [21]:
not_found_ids = [
    k
    for k in df.core_id.unique()
    if k not in set(found_keys)
]
len(not_found_ids)

51

In [22]:
df.assign(
    found_core_data=lambda x: ~x.core_id.isin(set(not_found_ids))
).head()

,unique_id,core_id,citing_title,citing_author,cited_title,cited_author,citation_context,citation_influence_label,citation_class_label,split,found_core_data
0,CC1,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Web search personalization with ontological us...,Sieg,They usually generate user models that describ...,0.0,5.0,train,True
1,CC2,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Exploring Scholarly Data with Rexplore,Osborne,The Computer Science Ontology (CSO)[3]is a lar...,0.0,0.0,train,True
2,CC3,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Klink-2: Integrating Multiple Web Sources to G...,Osborne,"In order to do so, we characterized all SN pub...",0.0,0.0,train,True
3,CC4,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Forecasting the Spreading of Technologies in R...,Osborne,"This API supports a number of applications, in...",1.0,0.0,train,True
4,CC5,158977742,Ontology-Based Recommendation of Editorial Pro...,Thiviyan Thanapalasingam,Supporting Springer Nature Editors by means of...,Osborne,It works according to three main steps:1) It r...,1.0,5.0,train,True


In [23]:
df_found = pd.DataFrame([
    d["data"]
    for d in core_data
    if d["status"] == "OK"
]).assign(core_id=lambda x: x["id"].astype(int))
df_found.head()

,id,authors,citations,contributors,datePublished,description,fullText,identifiers,publisher,relations,...,topics,types,year,doi,oai,downloadUrl,fulltextIdentifier,language,journals,core_id
0,158977742,"[Thanapalasingam, Thiviyan, Osborne, Francesco...",[],[],2018-10-28,Major academic publishers need to be able to a...,Ontology-Based Recommendation of Editorial Pro...,"[oai:oro.open.ac.uk:55516, 10.1007/978-3-030-0...",Springer,[http://iswc2018.semanticweb.org/],...,[],[],2018,10.1007/978-3-030-00668-6_21,oai:oro.open.ac.uk:55516,,NaN,NaN,NaN,158977742
1,211273841,"[Myers, Christina, Piccolo, Lara, Collins, Tre...",[],[],2018,This paper presents a participatory methodolog...,Open Research Online\nThe Open University’s re...,"[oai:oro.open.ac.uk:62141, 10.14236/ewic/HCI20...",NaN,[],...,[],[],2018,10.14236/ewic/HCI2018.164,oai:oro.open.ac.uk:62141,https://core.ac.uk/download/pdf/211273841.pdf,https://core.ac.uk/download/pdf/211273841.pdf,NaN,NaN,211273841
2,159766439,"[Pride, David, Knoth, Petr]",[],[],2018-09-30,Most Performance-based Research Funding System...,Open Research Online\nThe Open University’s re...,"[oai:oro.open.ac.uk:55743, 10.1007/978-3-030-0...",NaN,[],...,[],[],2018,10.1007/978-3-030-00066-0_17,oai:oro.open.ac.uk:55743,https://core.ac.uk/download/pdf/159766439.pdf,https://core.ac.uk/download/pdf/159766439.pdf,NaN,NaN,159766439
3,42702124,"[Escobar, Vanessa M., Delgado-Arias, Sabrina, ...",[],[],January 2015,The North American Carbon Program (NACP) was f...,S\nP\nM\nV\na\nb\nc\na\nK\nN\nC\nK\nK\nC\nC\n1...,"[oai:casi.ntrs.nasa.gov:20150023370, None]",NaN,[],...,[Environment Pollution],[],2015,NaN,oai:casi.ntrs.nasa.gov:20150023370,https://core.ac.uk/download/pdf/42702124.pdf,https://core.ac.uk/download/pdf/42702124.pdf,NaN,NaN,42702124
4,81765890,"[Henriette Golcher, Thomas B. Brunner, Helmut ...",[],[],2014,NaN,1 3\nOriginal article\nReceived: 11 January 20...,[10.1007/s00066-014-0737-7],Springer Nature,[http://dx.doi.org/10.1007/s00066-014-0737-7],...,[],[],2014,10.1007/s00066-014-0737-7,NaN,https://core.ac.uk/download/pdf/81765890.pdf,https://core.ac.uk/download/pdf/81765890.pdf,NaN,NaN,81765890


In [24]:
df_found[df_found.citations.apply(lambda x: len(x) != 0)].citations.shape

(14,)

In [25]:
df_found[df_found.topics.apply(lambda x: len(x) != 0)].topics.shape

(76,)

In [26]:
df_found[~df_found.description.isnull()].description.head().values

array(['Major academic publishers need to be able to analyse their vast catalogue of products and select the best items to be marketed in scientific venues. This is a complex exercise that requires characterising with a high precision the topics of thousands of books and matching them with the interests of the relevant communities. In Springer Nature, this task has been traditionally handled manually by publishing editors. However, the rapid growth in the number of scientific publications and the dynamic nature of the Computer Science landscape has made this solution increasingly inefficient. We have addressed this issue by creating Smart Book Recommender (SBR), an ontology-based recommender system developed by The Open University (OU) in collaboration with Springer Nature, which supports their Computer Science editorial team in selecting the products to market at specific venues. SBR recommends books, journals, and conference proceedings relevant to a conference by taking advantage of

In [27]:
df_found.year.value_counts()

2015    41
2014    33
2013    26
2012    24
2011    17
2010    13
2009     9
2018     4
2016     4
2008     4
2005     1
Name: year, dtype: int64

In [28]:
df_found.columns

Index(['id', 'authors', 'citations', 'contributors', 'datePublished',
       'description', 'fullText', 'identifiers', 'publisher', 'relations',
       'repositories', 'repositoryDocument', 'subjects', 'title', 'topics',
       'types', 'year', 'doi', 'oai', 'downloadUrl', 'fulltextIdentifier',
       'language', 'journals', 'core_id'],
      dtype='object')

In [29]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
ct = ColumnTransformer([
    ("citing_tfidf", TfidfVectorizer(), "citing_title"),
    ("cited_tfidf", TfidfVectorizer(), "cited_title"),
    ("citation_context_tfidf", TfidfVectorizer(),"citation_context"),
])
ct.fit(df)
df_features = ct.transform(df)
df_features.shape

(4000, 23045)

In [31]:
df_features

<4000x23045 sparse matrix of type '<class 'numpy.float64'>'
	with 204727 stored elements in Compressed Sparse Row format>

In [32]:
df_features[[0, 1, 5]]

<3x23045 sparse matrix of type '<class 'numpy.float64'>'
	with 93 stored elements in Compressed Sparse Row format>

In [33]:
TASKS={
    "purpose": "citation_class_label",
    "influence": "citation_influence_label"
}

def generate_data(df, label, split="train"):
    split_idx = df[(df.split == split)].index.tolist()
    X = df_features[split_idx]
    y = df.iloc[split_idx][label]
    print(f"{split}: X={X.shape}, y={y.shape}")
    return X, y, split_idx

def submission_pipeline(model, df, df_features, task, model_key=None, to_dense=False):
    label = TASKS[task]
    X_train, y_train, train_idx = generate_data(df, label, split="train")
    X_test, y_test, test_idx = generate_data(df, label, split="test")
    print(f"Training model")
    if to_dense:
        X_train = X_train.toarray()
        X_test = X_test.toarray()
    model.fit(X_train, y_train.astype(int))
    y_test = model.predict(X_test)
    print("Output label dist")
    print(pd.Series(y_test).value_counts())
    submission_file=f"{model_key}_{task}_submission.csv"
    print(f"Writing submission file: {submission_file}")
    df.iloc[test_idx][["unique_id"]].assign(**{label: y_test}).to_csv(submission_file, index=False)
    return model

# Gradient Boosting Classifier

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

In [35]:
for task in TASKS:
    model = GradientBoostingClassifier()
    submission_pipeline(model, df, df_features, task, model_key="gbt")

train: X=(3000, 23045), y=(3000,)
test: X=(1000, 23045), y=(1000,)
Training model
Output label dist
0    936
5     24
1     18
4     11
3      7
2      4
dtype: int64
Writing submission file: gbt_purpose_submission.csv
train: X=(3000, 23045), y=(3000,)
test: X=(1000, 23045), y=(1000,)
Training model
Output label dist
1    539
0    461
dtype: int64
Writing submission file: gbt_influence_submission.csv


# Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
for task in TASKS:
    model = RandomForestClassifier(n_jobs=-1)
    submission_pipeline(model, df, df_features, task, model_key="rf")

train: X=(3000, 23045), y=(3000,)
test: X=(1000, 23045), y=(1000,)
Training model
Output label dist
0    999
5      1
dtype: int64
Writing submission file: rf_purpose_submission.csv
train: X=(3000, 23045), y=(3000,)
test: X=(1000, 23045), y=(1000,)
Training model
Output label dist
1    501
0    499
dtype: int64
Writing submission file: rf_influence_submission.csv


# Multi Layer Neural Network

In [38]:
import sklearn
sklearn.__version__

'0.23.1'

In [39]:
from sklearn.neural_network import MLPClassifier

In [40]:
for task in TASKS:
    model = MLPClassifier(hidden_layer_sizes=(256,256,128))
    submission_pipeline(model, df, df_features, task, model_key="mlp", to_dense=False)

train: X=(3000, 23045), y=(3000,)
test: X=(1000, 23045), y=(1000,)
Training model
Output label dist
0    792
5    118
1     60
4     25
2      3
3      2
dtype: int64
Writing submission file: mlp_purpose_submission.csv
train: X=(3000, 23045), y=(3000,)
test: X=(1000, 23045), y=(1000,)
Training model
Output label dist
1    561
0    439
dtype: int64
Writing submission file: mlp_influence_submission.csv
